# Module 8 Assignment


A few things you should keep in mind when working on assignments:

1. Run the first code cell to import modules needed by this assignment before proceeding to problems.
2. Make sure you fill in any place that says `# YOUR CODE HERE`. Do not write your answer anywhere else other than where it says `# YOUR CODE HERE`. Anything you write elsewhere will be removed or overwritten by the autograder.
3. Each problem has an autograder cell below the answer cell. Run the autograder cell to check your answer. If there's anything wrong in your answer, the autograder cell will display error messages.
4. Before you submit your assignment, make sure everything runs as expected. Go to the menubar, select Kernel, and Restart & Run all. If the notebook runs through the last code cell without error message, you've answered all problems correctly.
5. Make sure that you save your work (in the menubar, select File → Save and CheckPoint).

-----

# Run Me First!

In [1]:
import sqlite3 as sql
import pandas as pd

from nose.tools import assert_equal, assert_true

-----

## Problem 1: Creating DB, Establishing a Connection, Creating a Table

In the code cell below, we declare a function named `create` that takes one function parameter: `file_path`, which is a string containing the file path to the SQLite3 database.

To complete this problem, finish writing the function `create`:
- establish a sqlite3 connection to the database 'file_path',
- create a cursor using the connection to this database,
- use the cursor to drop the `Courses` table if it exists,
- use the cursor to create the `Courses` table with the following columns:
    - `Subject` with type text and does not contain any NULL values,
    - `CourseNumber` with type integer and does not contain any NULL values,
    - `CRN` with type integer and does not contain any NULL values, and
- return the connection to the database and the database cursor.

-----

In [2]:
# connect to a datbase create a table
def create(file_path):
    '''
    Creates and establishes a connection to a database and 
    creates a table.
    
    Parameters
    ----------
    file_path: string containing path to create database
    
    Returns
    -------
    con: sqlite3 connection
    cur: sqlite3 database cursor object
    '''
    
    ###BEGIN SOLUTION
    con = sql.connect(file_path)
    cur = con.cursor() 
    query='DROP TABLE IF EXISTS Courses;'
    cur.execute(query)
    query = '''
    CREATE TABLE Courses (
        Subject TEXT NOT NULL,
        CourseNumber INT NOT NULL,
        CRN INT NOT NULL
    );
    '''
    cur.execute(query)
    return con, cur
    ###END SOLUTION###

In [3]:
con, cur = create('sql_files/m12.db')

cur.execute("PRAGMA table_Info('Courses')")
result = cur.fetchall()

ans = ''
for i in result:
    ans = ans + i[1] + " " + i[2] +'|'
    
assert_true('Subject TEXT' in ans, msg='Subject should be of type text')
assert_true('CourseNumber INT' in ans, msg='CourseNumber should be of type int')
assert_true('CRN INT' in ans, msg='CRN should be of type integer')


-----

## Problem 2: Inserting Data

In the code cell below, we declare a function named `insert` that takes two function parameter: `cur` which is a sqlite3 cursor and `data`, which is a list of tuples.

To complete this problem, finish writing the function `insert`:
- write and execute sql query to:
 - delete existing data in `Courses` table,
 - insert data in `data` to `Courses` table.

-----

In [4]:
data = [('ACCY', 199, 10033), ('ACCY', 199, 69998), ('ACCY', 200, 29670), ('ACCY', 201, 36478),
        ('ART', 102, 62794), ('ART', 150, 65459), ('ART', 310, 64968),
        ('IE', 300, 51898), ('IE', 512, 35414), ('IE', 360, 61503),
        ('LAW', 600, 30836), ('LAW', 604, 31954), ('LAW', 634, 56475)]
data

[('ACCY', 199, 10033),
 ('ACCY', 199, 69998),
 ('ACCY', 200, 29670),
 ('ACCY', 201, 36478),
 ('ART', 102, 62794),
 ('ART', 150, 65459),
 ('ART', 310, 64968),
 ('IE', 300, 51898),
 ('IE', 512, 35414),
 ('IE', 360, 61503),
 ('LAW', 600, 30836),
 ('LAW', 604, 31954),
 ('LAW', 634, 56475)]

In [5]:
def insert(cur, data):
    '''
    Delete existing data in Courses table and insert data in data to Courses table.
    
    Parameters
    ----------
    cur: sqlite3 cursor
    data: list of tuples
    
    Returns
    -------
    None
    '''
    
    ###BEGIN SOLUTION###
    cur.execute("DELETE FROM Courses")
    cur.executemany("INSERT INTO Courses (Subject, CourseNumber, CRN) VALUES (?, ?, ?)", data)
    ###END SOLUTION###

In [6]:
insert(cur, data)

cur.execute('select * from courses')
result = cur.fetchall()
assert_equal(result, data, msg="Your answer does not match the solution")

-----

## Problem 3: Selecting Data into a DataFrame

In the code cell below, we declare a function named `select` that takes two function parameters.

To complete this problem, finish writing the function `select`:
- Selects all columns of data from the `Courses` table for those courses with subject represented by function parameter "sub", and load to a dataframe
- return the dataframe.

-----

In [7]:
# Select data
def select(con, sub):
    '''
    Selects data from the courses table by subject.
    
    Parameters
    ----------
    con: sqlite3 connection
    sub: subject to select
    
    Returns
    -------
    dataframe that contains course info
    '''
    
    ###BEGIN SOLUTION###
    df = pd.read_sql("select * from courses where subject = '{}'".format(sub), con)
    return df
    ###END SOLUTION###

In [8]:
result = select(con, 'ACCY')
assert_equal(result.shape[0], 4, msg="Your answer does not match the solution")

-----

## Problem 4: Load and Create New Table

In the code cell below, we declare a function named `load_create` that takes two function parameters.

To complete this problem, finish writing the function `load_create`:
- Selects all columns of data from the `Courses` table and load to a dataframe
- write dataframe into a new database table-new_table_name
 - replace the table if already exists
 - **donot** write dataframe index to database

-----

In [9]:
def load_create(con, new_table_name):
    '''
    Load data from the Courses table to a dataframe, then write the dateframe to a new table.
    
    Parameters
    ----------
    con: sqlite3 connection
    new_table_name: name of the new table
    
    Returns
    -------
    None
    '''
    
    ###BEGIN SOLUTION###
    df = pd.read_sql("select * from courses", con)
    df.to_sql(new_table_name, con, index=False, if_exists='replace')
    return df
    ###END SOLUTION###

In [10]:
load_create(con, 'Courses_copy')
df_copy = pd.read_sql("select * from courses_copy", con)
assert_equal(df_copy.shape, (13,3), msg="Your function doesn't meet the requirments")
cur.close()
con.close()